In [6]:
import pandas as pd 
from functools import partial 
import ipywidgets as widgets
from IPython.display import display, clear_output
df = pd.read_csv('data/single_turn_trials_march_21.csv')

In [7]:
df = df[['EC Question', 'EC Answer', 'EC Answer Ordinal', 'JP Question', 'JP Answer', 'JP Answer Ordinal', 'TOM Question', 'TOM Answer', 'TOM Answer Ordinal', 'Game Description', 'Scenario']]

In [3]:
df = df[['EC Question', 'EC Answer', 'EC Answer Ordinal', 'JP Question', 'JP Answer', 'JP Answer Ordinal', 'TOM Question', 'TOM Answer', 'TOM Answer Ordinal', 'Game Description', 'Scenario']].fillna('')

In [17]:
# Generate widgets for all fields, treating ordinals as strings
text_area_widgets = {field: widgets.Textarea() for field in df.columns}

# Load current record into widgets
def load_record_to_widgets(index):
    record = df.iloc[index]
    for field, widget in text_area_widgets.items():
        widget.value = str(record[field])

# Update DataFrame from widgets
def update_record_from_widgets(index):
    for field, widget in text_area_widgets.items():
        df.at[index, field] = widget.value

current_index = 0

# Define a custom layout for the text areas to increase width and height
custom_layout = widgets.Layout(height='500px', width='100%')  # This sets the width to 100% of the parent container

# Apply the custom layout to all text area widgets
for widget in text_area_widgets.values():
    widget.layout = custom_layout

# For the horizontal box that contains the widgets, we also need to make sure it spans the full width
hbox_layout = widgets.Layout(width='100%')

# Update the display function to use this new hbox_layout
def display_edit_view(question_type=None):
    clear_output(wait=True)
    
    # Include navigation buttons in the edit view
    navigation_box = widgets.HBox([previous_button, next_button])

    if question_type:  # For Question, Answer & Answer Ordinal
        question_field = f"{question_type} Question"
        answer_field = f"{question_type} Answer"
        ordinal_field = f"{question_type} Answer Ordinal"
        edit_box = widgets.VBox([
            widgets.HBox([
                widgets.VBox([widgets.Label(value="Question:"), text_area_widgets[question_field]], layout=widgets.Layout(width='75%')),
                widgets.VBox([widgets.Label(value="Answer:"), text_area_widgets[answer_field]], layout=widgets.Layout(width='15%')),
                widgets.VBox([widgets.Label(value="Answer Ordinal:"), text_area_widgets[ordinal_field]], layout=widgets.Layout(width='10%'))
            ], layout=hbox_layout)
        ])
    else:  # For Scenario and Game Description
        field = question_type
        edit_box = widgets.VBox([
            widgets.Label(value=f"Edit {field}"),
            text_area_widgets[field]
        ], layout=hbox_layout)

    # Combine back button, edit box, and navigation buttons into one vertical box
    display(widgets.VBox([back_button, edit_box, navigation_box]))

# Function to determine the question type based on the index of the DataFrame
def get_question_type(index):
    # Assuming the question type is identifiable by a keyword in the column names
    for col in df.columns:
        if 'Question' in col and not df.at[index, col] == "":
            return col.replace(' Question', '')  # Returns 'EC', 'ToM', 'JP', etc.
    return None  # If no question type is found

# Adjust the next and previous functions
def go_next(b):
    global current_index
    update_record_from_widgets(current_index)
    if current_index < len(df) - 1:
        current_index += 1
        load_record_to_widgets(current_index)
        question_type = "EC"  # get the current question type
        display_edit_view(question_type)

def go_previous(b):
    global current_index
    update_record_from_widgets(current_index)
    if current_index > 0:
        current_index -= 1
        load_record_to_widgets(current_index)
        question_type = "EC" # get the current question type
        display_edit_view(question_type)

def display_main_view(b=None):
    clear_output(wait=True)
    load_record_to_widgets(current_index)
    display(navigation_box, widgets.VBox(buttons))

# Setup navigation buttons
next_button = widgets.Button(description="Next")
previous_button = widgets.Button(description="Previous")
back_button = widgets.Button(description="Back to Overview")
next_button.on_click(go_next)
previous_button.on_click(go_previous)
back_button.on_click(display_main_view)
navigation_box = widgets.HBox([previous_button, next_button])

# Generate buttons
buttons = [
    widgets.Button(description="Edit Scenario"),
    widgets.Button(description="Edit Game Description"),
    *[
        widgets.Button(description=f"Edit {qt} Q&A") for qt in ["EC", "TOM", "JP"]  # Assuming these are your question types
    ]
]

buttons[0].on_click(lambda b: display_edit_view(field="Scenario"))
buttons[1].on_click(lambda b: display_edit_view(field="Game Description"))
for i, qt in enumerate(["EC", "TOM", "JP"], start=2):
    buttons[i].on_click(lambda b, qtype=qt: display_edit_view(question_type=qtype))

# Initial display
load_record_to_widgets(current_index)
question_type = get_question_type(current_index)
display_edit_view(question_type)
    

In [18]:
df.to_csv('single_turn_trials_march_21_ectom_updated.csv', index=False)